In [1]:
# Standard libraries
import os
import json
import re
from datetime import datetime
from uuid import uuid4

# Third-party libraries
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from tqdm.auto import tqdm

# LangChain libraries
import langchain
from langchain.document_loaders import PyPDFLoader
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from dotenv import load_dotenv


load_dotenv()

# Access the OpenAI API key from environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise ValueError("OpenAI API key not found in environment variables")

# If you need to set it as an environment variable for other modules
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY


model_name = 'text-embedding-ada-002'

embeddings = OpenAIEmbeddings(
    model=model_name,
)

c:\Users\takat\.virtualenvs\Diploma-ESySJfXn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\takat\.virtualenvs\Diploma-ESySJfXn\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [12]:
llm_gpt3 = ChatOpenAI(
    model_name='gpt-3.5-turbo-1106',
    temperature=0.0
)
llm_gpt4 = ChatOpenAI(
    model_name='gpt-4',
    temperature=0.0
)

In [13]:
vectorstore_tad = FAISS.load_local('../faiss_db/tad-db', embeddings)
vectorstore_tad_title = FAISS.load_local('../faiss_db/tad-db-title-key', embeddings)

In [14]:
def generate_based_on_tad(query, retriever=vectorstore_tad, logging=False):
    qa_chain = load_qa_with_sources_chain(llm_gpt3, chain_type="stuff")
    qa_chain.document_prompt = PromptTemplate(
        input_variables=['course_title', 'course_credit', 'course_type', 'source', 'course_text'],
        template=(
            'BEGINING_OF_COURSE_DATA '
            'A kurzus címe: {course_title}, '
            'Kreditértéke: {course_credit}, '
            'Típusa: {course_type}, '
            'Forrás: {source}, '
            'A tárgy leírása: {course_text} '
            'END_OF_COURSE_DATA'
        )
    )
    qa_chain.llm_chain.prompt = PromptTemplate(
        input_variables=['summaries', 'question'],
        template=(
            'A tantárgyi adatlapokat felhasználva válaszolj a következő kérdésre. '
            'Lehetséges, hogy a megadott adatlap nem releváns a kérdésre, ebben az esetben ne használd fel. '
            'Ha nem tudod a választ, válaszolj úgy, hogy nem tudod. '
            'A tárgyakról az információk END_OF_COURSE_DATA és BEGINING_OF_COURSE_DATA között találhatóak. '
            'A típus értelmezésére példa: 4/2/0/v (előadás/gyakorlat/labor/vizsga). '
            'Ha "v" a vége, akkor a tárgy vizsgával zárul, ha "f", akkor félévközi. '
            'Egy félévben 30 kreditet kell teljesíteni. A tárgyak kreditértéke 1-15 között változhat. '
            'A tárgyak nevei a következők: {summaries} '
            'Ezeknek az információknak a birtokában válaszolj a következő kérdésre. '
            'Ha lehetőséged van, indokoljad meg, hogy miért ez a válaszod, de nem kötelező, csak ha érdemes: '
            '{question}'
        )
    )
    qa = RetrievalQA(
        combine_documents_chain=qa_chain,
        return_source_documents=logging,
        retriever=retriever.as_retriever(
            search_type="mmr",
            search_kwargs={'k': 5, 'fetch_k': 50}),
    )
    
    results = qa(query)
    if logging:
        print(results)
        
    return results['result']


In [15]:
print(generate_based_on_tad("Milyen tárgyakat oktat Mészáros Tamás?", retriever=vectorstore_tad))

A megadott adatlapok alapján Mészáros Tamás nem oktat egyetlen felsorolt tárgyat sem. Nem tudom, hogy melyik tárgyakat oktatja.


In [16]:
print(generate_based_on_tad("Javasolnál pár tárgyat ami véges automata témájú?"))

A véges automata témájú tárgyak javaslatához a "Fordítóprogramok a gyakorlatban" és az "Algoritmikus kérdések a bioinformatikában" tárgyakat javasolnám. Ezek a tárgyak tartalmazzák a véges automaták elméleti és gyakorlati alapjait, valamint bemutatják a véges automaták alkalmazásait a gyakorlatban. A "Fordítóprogramok a gyakorlatban" tárgyban a véges automaták használata a lexikális elemzés során kerül bemutatásra, míg az "Algoritmikus kérdések a bioinformatikában" tárgyban a véges automaták alkalmazása a szekvenciaillesztés és a szintaxis elemzés területén kerül bemutatásra. A véges automaták témájának mélyebb megértése és alkalmazása szempontjából ezek a tárgyak kiváló választások lehetnek.


In [17]:
print(generate_based_on_tad("Ki az analizis 1 tárgy oktatója?"))

Az analízis 1 tárgy oktatója Dr. Pataki Gergely, Dr. Bodrogné Dr. Réffy Júlia és Dr. Tasnádi Tamás.


In [18]:
print(generate_based_on_tad("Mi az ajánlott olvasmány Gajdos Sándor Adatbázisok tárgyához?"))

Az ajánlott olvasmány Gajdos Sándor Adatbázisok tárgyához az "S. Abiteboul, R. Hull, Victor Vianu: Foundations of Databases" című könyv. Ez azért ajánlott olvasmány, mert a tantárgy leírása szerint az adatbázisok elméleti megalapozásáról szól, és ez a könyv az adatbázisok alapjait és elméletét tárgyalja. A könyv segíthet a hallgatóknak megérteni az adatbázisok elméleti hátterét és alapjait, amelyekre a tantárgy épít.


In [19]:
print(generate_based_on_tad("Milyen tárgyat javasolnál ha mélytanulásról szeretnék tanulni?"))

A mélytanulás tárgyát javasolnám, mivel ez a tantárgy a Budapesti Műszaki és Gazdaságtudományi Egyetem Villamosmérnöki és Informatikai Karán elérhető, és részletesen bemutatja a mélytanulás elméletét és gyakorlati alkalmazásait. A tantárgy célja, hogy segítse a hallgatókat a modern mélytanuló szoftvereszközök és technikák elsajátításában és hatékony használatában. A tantárgy részletes tematikája számos fontos témakört érint, mint például a konvolúciós neurális hálózatok, rekurrens neurális hálózatok, önfelügyelt tanulás, gráf neurális hálózatok, és még sok más. A tantárgy elvégzéséhez átlagosan 150 tanulmányi munkaórát igényel, és a tantárgyfelelős személy Dr. Gyires-Tóth Bálint Pál, aki elismert szakértő a mélytanulás területén.


In [20]:
print(generate_based_on_tad("Hány kredit a Szoftvertervezés?", retriever=vectorstore_tad_title))

A Szoftvertervezés tantárgy kreditértéke 5.


In [21]:
print(generate_based_on_tad("Mi az ajánlott irodalom az Adatbázisok tárgyhoz?",  retriever=vectorstore_tad_title))

Az ajánlott irodalom az Adatbázisok tárgyhoz az alábbi:

S. Abiteboul, R. Hull, Victor Vianu: Foundations of Databases


In [22]:
print(generate_based_on_tad("Mi az ajánlott olvasmány Gajdos Sándor Adatbázisok tárgyához?",  retriever=vectorstore_tad_title))

Az ajánlott olvasmány az "S. Abiteboul, R. Hull, Victor Vianu: Foundations of Databases" című könyv. Ennek az az oka, hogy ez a könyv a tantárgy tematikájához kapcsolódóan a legrelevánsabb és legátfogóbb információkat tartalmazza az adatbázisokról.


In [34]:
compressor = LLMChainExtractor.from_llm(llm_gpt3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, 
    base_retriever=vectorstore_tad.as_retriever(
        search_type="mmr",
        search_kwargs={'k': 10, 'fetch_k': 50}
    )
)

compression_retriever.base_compressor.llm_chain.prompt = PromptTemplate(
    input_variables=['context', 'question',],
    template=(
        'Én egy BME diákoknak segítő chatbot vagyok. '
        'Felhasználva a tantárgyi adatlapokat, válaszolok a kérdésedre. '
        'Amennyiben a rendelkezésre álló adatlap nem releváns a kérdésedre, válaszom: "Nem tudom". '
        'Releváns dokumentumok esetén részletes és indokolt választ nyújtok. '
        'A tárgyak részletes adatai a BEGINING_OF_COURSE_DATA és END_OF_COURSE_DATA között találhatóak. '
        'Példa a típus értelmezésére: 4/2/0/v (előadás/gyakorlat/labor/vizsga). '
        'Vizsgával végződő tárgyak "v"-vel, félévközi tárgyak "f"-fel jelölve. '
        'Egy félévben 30 kredit teljesítése szükséges. '
        '\n\n'
        'A megadott szöveg: BEGINING_OF_COURSE_DATA  {context} END_OF_COURSE_DATA\n'
        'A kérdés: {question}'
    )
)

# compressed_docs = compression_retriever.get_relevant_documents("Milyen tárgyat javasolnál, ha mélytanulásról szeretnék tanulni?")


In [55]:
def compressed_tad_qa(query, logging=False):
    # Initialize the compressor with LLM GPT-3
    compressor = LLMChainExtractor.from_llm(llm_gpt3)

    # Set up the Contextual Compression Retriever
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, 
        base_retriever=vectorstore_tad.as_retriever(
            search_type="mmr",
            search_kwargs={'k': 10, 'fetch_k': 30}
        )
    )

    # Define the prompt template for the compressor
    compression_retriever.base_compressor.llm_chain.prompt = PromptTemplate(
        input_variables=['context', 'question'],
        template=(
            'A BME diákjainak segítő chatbot vagyok. '
            'Felhasználva a tantárgyi adatlapokat, válaszolok a kérdésedre. '
            'A tárgyak információi a BEGINING_OF_COURSE_DATA és END_OF_COURSE_DATA között találhatóak. '
            'Tárgyak típusának értelmezése: 4/2/0/v (előadás/gyakorlat/labor/vizsga). '
            'Vizsgával záruló tárgyak "v"-vel, félévközi tárgyak "f"-fel jelölve. '
            'Egy félévben 30 kredit teljesítése szükséges. '
            '\n\nA megadott szöveg: BEGINING_OF_COURSE_DATA {context} END_OF_COURSE_DATA\n '
            'Ha a megadott adatlap NEM releváns akkor a válaszom rövid, tömör. Azt mondom: "Nem tudom".'
            'Ha releváns, akkor részletes és indokolt választ nyújtok.'
            'A kérdés: {question}'
        )
    )
    # Retrieve and combine compressed documents
    compressed_docs = compression_retriever.get_relevant_documents(query)
    # Format and join the documents with their metadata
    docs = '\n\n'.join([
        'BEGINING_OF_COURSE_DATA '
        f'A kurzus címe: {doc.metadata.get("course_title", "Ismeretlen cím")}, '
        f'Kreditértéke: {doc.metadata.get("course_credit", "Ismeretlen kreditérték")},'
        f'Típusa: {doc.metadata.get("course_type", "Ismeretlen típus")}, '
        f'Forrás: {doc.metadata.get("source", "Ismeretlen forrás")}, '
        f'A tárgy leírása: {doc.page_content} '
        'END_OF_COURSE_DATA'
        for doc in compressed_docs
    ])

    # Optional logging to print each document's content and metadata
    if logging:
        for doc in compressed_docs:
            print(
               'BEGINING_OF_COURSE_DATA '
                f'A kurzus címe: {doc.metadata.get("course_title", "Ismeretlen cím")}, '
                f'Kreditértéke: {doc.metadata.get("course_credit", "Ismeretlen kreditérték")},'
                f'Típusa: {doc.metadata.get("course_type", "Ismeretlen típus")}, '
                f'Kurzuskód: {doc.metadata.get("course_id", "Ismeretlen id")}, '
                f'A tárgy leírása: {doc.page_content} '
                'END_OF_COURSE_DATA'
            )
    # Set up the ChatPromptTemplate for response generation
    qa = ChatPromptTemplate.from_template(
        template=(
            'A BME diákjainak segítő chatbot vagyok. '
            'Kombináld az alábbi releváns válaszokat a felhasználó kérdésére. '
            'Jelmagyarázat: 4/2/0/v (előadás/gyakorlat/labor/vizsga). '
            'Egy félévben 30 kreditet kell teljesíteni. '
            'Az adatok a BEGINING_OF_COURSE_DATA és END_OF_COURSE_DATA között találhatóak.'
            'A válaszomhoz a lehető legtöbb információt felhasználom. '
            'Lehet, hogy egy adatlap nem releváns a kérdésre, ebben az esetben nem használom fel. '
            '\nAz eddig összegyűjtött információ:\n' + docs + 
            '\nA felhasználó eredeti kérdése: ' + query + 
            '\nA végső válaszod:'
        )
    )

    # Chain the QA template with the LLM for response generation
    chain = qa | llm_gpt4
    
    # Invoke the chain to generate the final response
    return chain.invoke({}).content


In [56]:
result = compressed_tad_qa("Milyen tárgyat javasolnál ha mélytanulásról szeretnék tanulni?", logging=True)

c:\Users\takat\.virtualenvs\Diploma-ESySJfXn\Lib\site-packages\langchain\chains\llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


BEGINING_OF_COURSE_DATA A kurzus címe: Mélytanulás, Kreditértéke: 5,Típusa: 2/1/0/v, Kurzuskód: VITMMA19, A tárgy leírása: A mélytanulás tantárgy mellett ajánlott tárgy a Gépi tanulás. END_OF_COURSE_DATA
BEGINING_OF_COURSE_DATA A kurzus címe: Adatelemzés mélytanulási módszerekkel, Kreditértéke: 5,Típusa: 2/2/0/v, Kurzuskód: VITMAC15, A tárgy leírása: A "Adatelemzés mélytanulási módszerekkel" tantárgyat javasolnám, mivel a tantárgy célja az adatelemzéshez szükséges gépi tanulás és azon belül a mélytanulás módszereinek oktatása. A tantárgy során ismertetett módszerek strukturált gazdasági adatok előrejelzésén és strukturálatlan multimédia tartalmak elemzésén keresztül tekintik át, így részletesen megismerkedhetsz az intelligens adatelemzéstől a mélytanulásig. END_OF_COURSE_DATA
BEGINING_OF_COURSE_DATA A kurzus címe: Deep Learning a gyakorlatban Python és LUA alapon, Kreditértéke: 4,Típusa: 2/2/0/v, Kurzuskód: VITMAV45, A tárgy leírása: A mélytanulás (Deep Learning) témakörében a Budapest

In [57]:
print(result)

A mélytanulás témakörében több tantárgyat is ajánlanék. Az egyik a "Mélytanulás" nevű kurzus, amely 5 kreditértékű és 2/1/0/v típusú. A mélytanulás tantárgy mellett ajánlott tárgy a Gépi tanulás. 

Ezen kívül a "Adatelemzés mélytanulási módszerekkel" tantárgyat is javasolnám, mivel a tantárgy célja az adatelemzéshez szükséges gépi tanulás és azon belül a mélytanulás módszereinek oktatása. Ez a kurzus is 5 kreditértékű és 2/2/0/v típusú.

A "Deep Learning a gyakorlatban Python és LUA alapon" tantárgy 4 kreditértékű és 2/2/0/v típusú. Ez a tantárgy részletesen foglalkozik a mély tanuló rendszerek tervezésével, megvalósításával és az órán tanult eszközök használatával.

A "Deep learning alkalmazása a vizuális informatikában" nevű tantárgy a gépi látás területén alkalmazza a GPU alapú deep learning technikákat, és bemutatja a képi információfeldolgozás és a látás alapú robotika feladatait. Ez a kurzus 4 kreditértékű és 2/0/2/v típusú.

Végül a "Deep-learning a vizuális informatikában" tant